# SKT-koBERT

https://github.com/SKTBrain/KoBERT

## Install & Library

In [54]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import os

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)})

# CPU or GPU
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
device

'cuda:1'

In [57]:
from scipy.special import softmax
pd.set_option('display.float_format', '{:.5f}'.format)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


## Path

In [17]:
model_path = '/home/hyejeongeun/Todays_Chatbot/Model/Model/'
review_path = '/home/hyejeongeun/Todays_Chatbot/Data/Review/Review_embedding/'

## Model & Tokenizer

In [3]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [4]:
# 내장된 tokenizer와 sentencepiece 함수를 쓰는 듯
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        #self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i]) #+ (self.labels[i], ))

    def __len__(self):
        return (len(self.sentences))

In [6]:
## Setting parameters
max_len = 210 #210 #수정
batch_size = 16 # 64 #수정
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, #2 #수정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        
        classifier = self.classifier(pooler)
        
        if self.dr_rate:
            out = self.dropout(classifier)
  
        return out

In [12]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [13]:
model.load_state_dict(torch.load(model_path+'epoch_5_qna_sep_model_4.pt'))

<All keys matched successfully>

In [14]:
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

## score & classification 출력하게 수정

In [31]:
def review_BERTClassifier(_dataloader):
    for batch_id, (token_ids, valid_length, segment_ids) in enumerate(_dataloader):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length  

        probability = model.forward(token_ids, valid_length, segment_ids)
        probability = np.array(probability.tolist())

        if batch_id == 0:
            review_probability = probability
        else:
            review_probability = np.concatenate([review_probability, probability], axis=0)

        del probability
        torch.cuda.empty_cache() # GPU 캐시 삭제
        
    # max index 추출
    review_probability = softmax(review_probability, axis=1)
    review_class = np.argmax(review_probability, axis=1)
    score = np.max(review_probability, axis=1)
    
    return review_class, score

## Review Classification

In [18]:
df = pd.read_csv(review_path+'review_388715_embedding.csv', sep=',')

In [19]:
tokens = df['comment_mecab'].tolist()

data = BERTDataset(tokens, 0, tok, max_len, True, False)
_dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, num_workers=5)

In [32]:
# classifier
for i in tqdm_notebook(range(1)):
    review_class, score = review_BERTClassifier(_dataloader)

/home/hyejeongeun/.conda/envs/hyejeongeun/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
# csv 파일로 내보내기
df['score'] = score.tolist()
df['review_class_new'] = review_class.tolist()

In [34]:
# 기존 결과랑 변한거 없는지 확인
df[df['review_class']!=df['review_class_new']]

,created_at,writer_id,praise_count,prod_id,prod_name,prod_opt,comment,image_url,comment_spellcheck,comment_mecab,review_embedding,review_class,score,review_class_new


In [63]:
df['review_class'].value_counts()

1    13773
0    9848 
2    21   
3    6    
Name: review_class, dtype: int64

In [36]:
df_1 = df[['comment','score','review_class']]

In [ ]:
"""
상품-0: 다양한 연출이 가능하고 이리저리 옮겨가며 쓰기 좋아서 굉장히 만족합니당!!!! 망가짐없이 아직도 잘 쓰고 있어요! 0.99550
배송-1: 3월 11일 낮에 주문해서 24일에 왔습니다... 제품은 마음에 드는데 배송 너무 오래걸렸네요ㅠㅠ 0.97133
교환-2: 불량을 두번이나 보내줘서 짜잉났던제품...그치만 빠르게 대처해주셔서 어렵게 완성된 제품이네효 이쁘긴합니다! 0.61859
환불-3: 환불해주세요 나사 돌리자마자 쩍 소리가 나더니\n부러졌습니다 0.93713
반품-4: X
"""

In [58]:
df_1[df_1['review_class']==0].sort_values(by='score', ascending=False)[:20]

,comment,score,review_class
7037,다양한 연출이 가능하고 이리저리 옮겨가며 쓰기 좋아서 굉장히 만족합니당!!!! 망가짐없이 아직도 잘 쓰고 있어요!,0.99550,0
21033,연약해보이지만 단단하게 잘조이면\n튼튼한것같기도 해요!,0.99550,0
548,연장이 없어도 조립이 가능하나\n손바닥이 너무 아파요 !!,0.99550,0
23576,연휴에 시켜서 배송이 많이 늦었지만 역시 이뻐요!! 근데 조립하기가 조금 힘에겹네요 ㅠ,0.99550,0
11979,"연휴때문에 배송느릴거라 생각했는데 너무 빨리와서 놀랬어요,,, 조립도 엄청 간단하고 디자인도 깔끔하니 넘 이뻐요 짱짱",0.99550,0
6554,연결부분이 끊어졌더라구요. \n그래서 반품 할까하다가 지지하는 부분이 아니라 \n그냥 씁니다.\n물건 확인 잘하시고 보내주시면 좋겠어요,0.99550,0
21591,연결나사가 육각렌치...!!ㅜㅜ 손힘이 필요하긴 하지만 가성비로 보나 디자인으로 보나 좋습니다. 아주 만족스럽게 사용중이에요!,0.99550,0
21423,연결부위가 부러져서 왔네요\r\n다 조립하고 알아서.... \r\n목공풀로 붙여서 붙긴했는데 또 언제 떨어질지 불안하지만...조심히 사용중입니다,0.99550,0
1474,연휴전에 받아보앗지만 불량으로 다시보내주셔서 지금에야 구매확정을 해요ㅎㅎ 통화주셔서 너무 친절하게 해주시구 만족합니당! 제품은 너무 조아유👍🏻,0.99550,0
7935,푹신푹신 좋아요~~ 잘 사용할께요^^,0.99546,0


In [59]:
df_1[df_1['review_class']==1].sort_values(by='score', ascending=False)[:20]

,comment,score,review_class
2636,호우.... 조립되게 쉽고 좋네용 옛날부터 사야지 사야지 하다가 이번기회에 삿어용 얼른 집꾸며야지><,0.97405,1
3779,호야화분을 올려놓을려고 샀습니다. 사이즈 딱 제가 원하던거구요 이뻐요 근데 조립이 너무너무 힘들었습니다..\n걍 일반 나사? 였으면 더 편했을거 같아요.....\n그래도 사이즈랑 디자인은 이뻐요~,0.97405,1
11097,월요일에 주문하고 수요일에 받았습니다! 배송도 빠르고 제품도 깔끔하고 너무 예쁩니다! 조립은 아빠가 해주셨만 어렵지 않았어요. 잘쓰겠습니다,0.97133,1
18523,5월 24일에 사전예약 상품인 줄 모르고 주문해서 \n거의 2주가까이 기다려받았어요\n기다린 보람이 있네요ㅠ\n하나 더 주문할까해요ㅠ이쁨...!!!,0.97133,1
22483,2월24일에 주문했는데 3월8일에 왔어요 배송 별점 한개주는것도 아까워요,0.97133,1
15451,"2월 24일에 주문해서 3월 7일에 받았어요! 그래도 오늘의 특가로 엄청 저렴하게 16,000원대에 원목 사이드테이블을 구매했습니다. 기다린 보람이 있는만큼 디자인은 예쁘네요. 나사를 조립하는건 다른 원목제품 나사조립보단 힘이 많이 들어가네요. 침대 옆에 온수냉배드 사용할때 필요한 보일러통을 둘것이 필요해서 구매했는데 딱 입니다! 잘 사용하겠습니다.",0.97133,1
7414,3월 11일 낮에 주문해서 24일에 왔습니다... 제품은 마음에 드는데 배송 너무 오래걸렸네요ㅠㅠ,0.97133,1
7583,2월28일에 주문했는데 오늘받았네염.... 아무것도모르고 마냥 기다렸네요 출고가 늦어지면 연락이라도 한번주시지.... 육각렌치 돌리기도 넘 빡세요 ㅠ,0.97133,1
12756,9월9일이전은 추석전배송이라했는데 ..^^어제왔어요,0.97133,1
23540,3월2일에결재해서15일에받았네요..배송관련해서궁금해이틀에걸쳐7번정도전화했으나한번도연결되지못하고연락한통없는등서비스는완전엉망이네요 제품은그냥가격값이랑비슷하네요 또구매할생각은없습니다,0.97133,1


In [60]:
df_1[df_1['review_class']==2].sort_values(by='score', ascending=False)[:20]

,comment,score,review_class
1097,불량이 있었지만 빠른 교환처리 만족합니다~,0.61859,2
1430,불량 왔어요.. 나사가 헛 돌아서 힘들어 돌아가시는줄.,0.61859,2
23023,불량이와서 나사가 다안들어가요... 두군데가 그렇ㄱ습니다 환불이나 교환귀찮어서 그냥 대충사용중입니다..ㅎㅎ,0.61859,2
21231,불량상품이 와서 반품하고 다시 주문한건데 조립할때 구멍들을 맞추느라 애먹었네요 해놓고 나니 이쁘긴 합니다,0.61859,2
20850,불도 잘들어오고 분위기 있게 만들어줘요 ^^,0.61859,2
18802,불량제품이 와서 교환하는데 너무 오래걸렸어요 가격대비 디자인도 이쁘고 쓸만하네요,0.61859,2
18557,불량 왔어요 못으로 조이는 한부분이 끝까지 조여지지 않고 억지로 힘으로 조이려고 하면 부서지려고 해서 그냥 저렇게 벌어진 상태로 두고 쓰는 중이에요 이미 뜯은거 겨환 환불도 귀찮고. 디자인 인정. 근데 배송 엄청 느림,0.61859,2
18072,불량이와서 그부분은 빠르게처리해주셨네요 ㅎㅎ\n이뻐요 잘쓰고있습니다~^^,0.61859,2
16734,불량이 왓는지 모서리 쪽이 안맞 ... 조립도 어려웠어요,0.61859,2
16081,불량을 두번이나 보내줘서 짜잉났던제품...그치만 빠르게 대처해주셔서 어렵게 완성된 제품이네효 이쁘긴합니다!,0.61859,2


In [61]:
df_1[df_1['review_class']==3].sort_values(by='score', ascending=False)[:20]

,comment,score,review_class
3140,이걸 돈 받고 팔다니 신기하다 신기해,0.95759,3
14094,돈아깝다는 생각 하나도 안들고 엄청 예뻐요ㅎㅎ,0.95759,3
23619,환불해주세요 나사 돌리자마자 쩍 소리가 나더니\n부러졌습니다,0.93713,3
6825,값싸게 고급진 느낌 낼려면 강추해요 조립도 쉬워요,0.35955,3
8872,싼 값에 샀는데 잘 쓰고 있어서 만족합니다. 디자인이 이뻐요,0.35955,3
5249,액자 예뻐요!!!!! 또 살거에요 비싸긴해요..,0.34339,3


In [62]:
df_1[df_1['review_class']==4].sort_values(by='score', ascending=False)[:20]

,comment,score,review_class
